In [205]:
import pandas as pd 
import numpy as np

from pathlib import Path

In [206]:
path_hub = "sample-input/hubspot-crm-exports-sync-erwin-hubspot-2023-10-16.xlsx"
path_er = "sample-input/crm.lead.csv"
parent_dir = Path.cwd().parents[0]

try:
    df_hub = pd.read_csv(parent_dir / path_hub)
except UnicodeDecodeError:
    df_hub = pd.read_excel(parent_dir / path_hub)

try:
    df_er = pd.read_csv(parent_dir / path_er)
except UnicodeDecodeError:
    df_er = pd.read_excel(parent_dir / path_er)


/home/anj/miniconda3/envs/streamlit/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Clean DF

In [207]:
def clean_email(df, email_col):
    return df[email_col].str.strip().str.lower()

def clean_phone(df, phone_col):
    return (df[phone_col]
            .str.strip()
            .str.replace("/D", "", regex=True)
            .str.replace("^62", "0", regex=True)
            .str.replace("^8", "08", regex=True)
            )

In [208]:
def clean_df_hub(df):
    return (df
        .rename(columns=lambda c: c.lower().replace(" ", "_"))
        .assign(
            email=lambda df_: clean_email(df_, "email"),
            phone_number=lambda df_: clean_phone(df_, "phone_number"),
        )
        .loc[lambda df_: (df_["email"] != np.nan) & (df_["phone_number"] != np.nan)]
        .drop_duplicates(subset=["email"])
        .drop_duplicates(subset=["phone_number"])
        .dropna(subset=["phone_number"])
        .dropna(subset=["email"])
    )


def clean_df_er(df):
    map_stage = {
        "Appointment": "Sales qualified lead",
        "Show": "Opportunity",
        "Down Payment": "Customer",
        "Fully Paid": "Customer",
        "False": "Lead",
        "Lead": "Lead",
        np.nan: "Lead",
    }
    map_lp = {
        "Ya": "Yes",
        "Mau!": "Yes",
        "Tidak": "No",
        "Engga, deh.": "No",
        np.nan: "Blank",
    }

    return (df
        .rename(columns=lambda c: c.lower().replace(" ", "_").replace("/", "_").replace("?", ""))
        .rename(columns={
            "stage_display_name": "stage"
        })
        .loc[
            lambda df_: 
                (df_["source"].isin(["Digital-Paid", "Digital-Organic"])) &\
                (df_["stage"] != "Renewal") &\
                (df_["email"] != np.nan) &\
                (df_["phone"] != np.nan)
        ]
        .assign(
            email=lambda df_: clean_email(df_, "email"),
            phone=lambda df_: clean_phone(df_, "phone"),
            stage=lambda df_: df_["stage"].map(map_stage),
            learning_preference=lambda df_: df_["learning_preference"].map(map_lp),
            tmk_call=lambda df_: df_["tmk_call"].map({True: 1, False: 0})
        )
        .drop_duplicates(subset=["email"])
        .drop_duplicates(subset=["phone"])
        .dropna(subset=["email"])
        .dropna(subset=["phone"])
    )

In [209]:
df_hub_clean =clean_df_hub(df_hub)
df_er_clean =clean_df_er(df_er)

# Merge

In [210]:
df_merge_email = (df_hub_clean
    .merge(
        df_er_clean, 
        how="inner", 
        left_on="email", 
        right_on="email", 
        validate="one_to_one"
    )
    .loc[:, ["record_id", "phone_number", "email", "stage", "learning_preference", "tmk_call"]]
)

df_merge_phone = (df_hub_clean
    .loc[~df_hub_clean["email"].isin(df_merge_email["email"])]
    .merge(
        df_er_clean, 
        how="inner", 
        left_on="phone_number", 
        right_on="phone", 
        validate="one_to_one"
    )
    .loc[:, ["record_id", "phone_number", "email_x", "stage", "learning_preference", "tmk_call"]]
    .rename(columns={"email_x": "email"})
)

df_match = (pd.concat([df_merge_email, df_merge_phone], axis=0)
    .dropna(subset=["stage", "learning_preference", "tmk_call"])
    .rename(columns=lambda c:(c
                              .title()
                              .replace("_", " ")
                              .replace("Id", "ID")
                              .replace("Stage", "Lifecycle Stage")
                              .replace("Tmk Call", "Is TMK Call?")
                              ))
)
df_match.head()

,Record ID,Phone Number,Email,Lifecycle Stage,Learning Preference,Is TMK Call?
0,307703451,081266437772,blackleaner@gmail.com,Lead,Yes,1
1,307623101,+6282291997772,acsflorist@gmail.com,Lead,Blank,0
2,307702451,081511290713,novitanazwa102@gmail.com,Lead,Yes,1
3,307700451,0895110031098,14out11@gmail.com,Lead,Yes,1
4,307698401,089686946541,wildaamelina96@gmail.com,Lead,Yes,1


# Get Unmatch

In [216]:
df_no_match = (df_er_clean
    .loc[
        ~(df_er_clean["email"].isin(df_match["Email"])) &\
        ~(df_er_clean["phone"].isin(df_match["Phone Number"])), 
        ["email", "phone", "stage", "learning_preference", "tmk_call"]
    ]
    .rename(columns=lambda c:(c
                              .title()
                              .replace("_", " ")
                              .replace("Id", "ID")
                              .replace("Stage", "Lifecycle Stage")
                              .replace("Tmk Call", "Is TMK Call?")
                              ))
)
df_no_match.head()

,Email,Phone,Lifecycle Stage,Learning Preference,Is TMK Call?
23,riskitadwsptyn139@gmail.com,081932397843,Lead,Yes,1
29,humairahoseki98@gmail.com,0895605917523,Sales qualified lead,Yes,1
53,nia.rachmania@gmail.com,087783871151,Sales qualified lead,Blank,0
60,bacaanindri@gmail.com,081318714331,Lead,Yes,1
68,putrimonicafebiana@gmail.com,089601019170,Lead,No,0
